In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import yaml
with open('../setting/path.yaml', 'r') as file:
    config = yaml.safe_load(file)

# パスの設定
survey_path_2024_q2 = config['survey_data_path_2024_q2']
survey_path_2024_q3 = config['survey_data_path_2024_q3']
column_list_path = config['column_mapping']
# 出力
survey_2024_Q2andQ3_processed = config['survey_data_2024_processed']

## 個票データの読み込み

In [ ]:
df_q2 = pd.read_csv(survey_path_2024_q2,
                    usecols=list(range(88)))

df_q3 = pd.read_csv(survey_path_2024_q3,
                    usecols=list(range(88)))

In [ ]:
df_q2.shape

In [ ]:
df_q3.shape

In [ ]:
df_q2.columns

In [ ]:
assert df_q2.columns.equals(df_q3.columns), '2つの変数名が一致していません。'

## カラム名の変更

In [ ]:
column_list = pd.read_excel(column_list_path)
column_list.shape

In [ ]:
def rename_columns(df, column_list):
    # カラム名の変更
    column_mapping = dict(zip(column_list['変数名'], column_list['変数ラベル']))
    # データフレームのカラム名を更新
    df = df.rename(columns=column_mapping)
    assert set(column_list['変数ラベル']) == set(df.columns), "カラム名が正しく変更されていません"
    
    return df

In [ ]:
df_q2 = rename_columns(df_q2, column_list)
df_q3 = rename_columns(df_q3, column_list)

In [ ]:
df_q2.columns

In [ ]:
df_q2.isna().sum()

In [ ]:
df_q3.isna().sum()

## 調査港（17区分）のコード値の設定

In [ ]:
def prot_mapping_data(df):
    # 調査港の変換マッピング
    port_mapping = {
        'SPK': '新千歳空港',
        'HKD': '函館空港',
        'SDJ': '仙台空港',
        'HND': '東京国際空港（羽田空港）',
        'NRT': '成田国際空港',
        'KMQ': '小松空港',
        'FSZ': '富士山静岡空港',
        'NGO': '中部国際空港',
        'KIX': '関西国際空港',
        'HIJ': '広島空港',
        'JPSHS': '関門港（下関）',
        'TAK': '高松空港',
        'FUK': '福岡空港',
        'JPHKT': '博多港',
        'JPIZH': '厳原港',
        'KOJ': '鹿児島空港',
        'OKA': '那覇空港'
    }
    
    # 変換の実行
    df['調査港（17区分）'] = df['調査港（17区分）'].map(port_mapping)
    
    # 変換結果の確認
    assert df['調査港（17区分）'].isnull().sum() == 0, "変換に失敗した値があります"

    return df

In [ ]:
df_q2 = prot_mapping_data(df_q2)
df_q3 = prot_mapping_data(df_q3)

## 日付型の変換

In [ ]:
def to_date_survey(df):
    # 調査日と入国日のカラムを日付型に変換
    df['調査日'] = pd.to_datetime(df['調査日'], format='%m/%d/%Y')
    df['入国日'] = pd.to_datetime(df['入国日'], format='%m/%d/%Y')
    
    # 変換後のデータ型を確認
    assert pd.api.types.is_datetime64_any_dtype(df['調査日']), "調査日のカラムが日付型に変換されていません。"
    assert pd.api.types.is_datetime64_any_dtype(df['入国日']), "入国日のカラムが日付型に変換されていません。"
    
    print("調査日と入国日のカラムは正しく日付型に変換されました。")

    return df

In [ ]:
df_q2 = to_date_survey(df_q2)
df_q3 = to_date_survey(df_q3)

## 「滞在日数（7区分）」の変換

In [ ]:
def stay_dura_mapping(df):
    # 滞在日数の変換マッピング
    stay_duration_mapping = {
        1: '3日間以内',
        2: '4～6日間',
        3: '7～13日間',
        4: '14～20日間',
        5: '21～27日間',
        6: '28～90日間',
        7: '91日間以上1年未満'
    }
    
    # 変換の実行
    df['滞在日数（7区分）'] = df['滞在日数（7区分）'].map(stay_duration_mapping)
    
    # 変換結果の確認
    assert df['滞在日数（7区分）'].isnull().sum() == 0, "変換に失敗した値があります"
    
    return df

In [ ]:
df_q2 = stay_dura_mapping(df_q2)
df_q3 = stay_dura_mapping(df_q3)

## 国籍・地域を取得

In [ ]:
# 国籍・地域コードのExcelファイルを読み込み
nationality_list = pd.read_excel(config['nationality_region_mapping'])

def nationality_mapping_data(df, nationality_list):
    # 国籍・地域コードと国名のマッピングを作成
    country_mapping = pd.Series(nationality_list['国籍・地域'].values,
                                index=nationality_list['国籍・地域コード']).to_dict()
    
    # 国籍・地域コードを国籍・地域名に変換して新しいカラムを作成
    df['国籍・地域'] = df['国籍・地域コード'].map(country_mapping)
    assert df['国籍・地域'].isnull().sum() == 0, "一部の国籍・地域コードがマッピング表に存在しません"
    
    df = df.drop(columns=['国籍・地域コード'], axis=1)
    
    return df

In [ ]:
df_q2 = nationality_mapping_data(df_q2, nationality_list)
df_q3 = nationality_mapping_data(df_q3, nationality_list)

In [ ]:
df_q2['国籍・地域'].value_counts()

In [ ]:
df_q3['国籍・地域'].value_counts()

## 国籍･地域（21区分）と居住国･地域（21区分）を変換

In [ ]:
# 国籍・地域コードと国名のマッピング
country_code_mapping = {
    1: "韓国", 2: "台湾", 3: "香港", 4: "中国", 5: "タイ", 
    6: "シンガポール", 7: "マレーシア", 8: "インドネシア", 9: "フィリピン",
    10: "ベトナム", 11: "インド", 12: "英国", 13: "ドイツ",
    14: "フランス", 15: "イタリア", 16: "スペイン",
    17: "ロシア", 18: "米国", 19: "カナダ",
    20: "オーストラリア", 21: "その他"
}

# 国籍・地域（21区分）と　居住国･地域（21区分）を 国籍・地域名に変換する
df_q2['国籍・地域（21区分）'] = df_q2['国籍・地域（21区分）'].map(country_code_mapping)
df_q2['居住国･地域（21区分）'] = df_q2['居住国･地域（21区分）'].map(country_code_mapping)
df_q3['国籍・地域（21区分）'] = df_q3['国籍・地域（21区分）'].map(country_code_mapping)
df_q3['居住国･地域（21区分）'] = df_q3['居住国･地域（21区分）'].map(country_code_mapping)

# 変換後のデータの確認
assert df_q2['国籍・地域（21区分）'].isnull().sum() == 0, "一部の国籍・地域がマッピング表に存在しません"
assert df_q2['居住国･地域（21区分）'].isnull().sum() == 0, "一部の居住国・地域がマッピング表に存在しません"
assert df_q3['国籍・地域（21区分）'].isnull().sum() == 0, "一部の国籍・地域がマッピング表に存在しません"
assert df_q3['居住国･地域（21区分）'].isnull().sum() == 0, "一部の居住国・地域がマッピング表に存在しません"

## 居住地・地域の変換

In [ ]:
# 国籍・地域コードのExcelファイルを読み込み
country_list = pd.read_excel(config['country_code_mapping'])

def country_mapping_data(df, country_list):
    # 国籍・地域コードと国名のマッピングを作成
    country_mapping = pd.Series(country_list['居住国・地域'].values,
                                index=country_list['居住国・地域コード']).to_dict()
    
    # 国籍・地域コードを国籍・地域名に変換して新しいカラムを作成
    df['居住国・地域'] = df['居住国・地域コード'].map(country_mapping)
    assert df['居住国・地域'].isnull().sum() == 0, "一部の居住国・地域コードがマッピング表に存在しません"
    
    df = df.drop(columns=['居住国・地域コード'], axis=1)

    return df

In [ ]:
df_q2 = country_mapping_data(df_q2, country_list)
df_q3 = country_mapping_data(df_q3, country_list)

## 中国居住地（15区分）の変換

In [ ]:
# 中国居住地のマッピング
china_residence_mapping = {
    1: "北京市",
    2: "上海市",
    3: "重慶市",
    4: "天津市",
    5: "広東省",
    6: "山東省",
    7: "遼寧省",
    8: "浙江省",
    9: "江蘇省",
    10: "四川省",
    11: "マカオ",
    12: "湖北省",
    13: "福建省",
    14: "陝西省",
    15: "その他",
    -98: "不明",
    -99: "非該当"
}

# DataFrameのカラム名 '中国居住地（15区分）' を使用して居住地を変換する
df_q2['中国居住地（15区分）'] = df_q2['中国居住地（15区分）'].map(china_residence_mapping)
df_q3['中国居住地（15区分）'] = df_q3['中国居住地（15区分）'].map(china_residence_mapping)

# 変換後のデータの確認
assert df_q2['中国居住地（15区分）'].isnull().sum() == 0, "一部の中国居住地コードがマッピング表に存在しません"
assert df_q3['中国居住地（15区分）'].isnull().sum() == 0, "一部の中国居住地コードがマッピング表に存在しません"

## 主な来訪目的（16区分）の変換

In [ ]:
def visit_mapping_data(df):
    # 主な来訪目的のマッピング
    visit_purpose_mapping = {
        1: "観光・レジャー",
        2: "親族・知人訪問",
        3: "ハネムーン",
        4: "学校関連の旅行",
        5: "スポーツ・スポーツ観戦",
        6: "イベント",
        7: "留学",
        8: "治療・検診",
        9: "インセンティブツアー",
        10: "展示会・見本市",
        11: "国際会議",
        12: "企業ミーティング",
        13: "研修",
        14: "その他ビジネス",
        15: "トランジット",
        16: "その他"
    }
    
    # DataFrameのカラム名 '主な来訪目的（16区分）' を使用して目的を変換する
    df['主な来訪目的（16区分）'] = df['主な来訪目的（16区分）'].map(visit_purpose_mapping)
    
    # 変換後のデータの確認
    assert df['主な来訪目的（16区分）'].isnull().sum() == 0, "一部の来訪目的コードがマッピング表に存在しません"

    return df

In [ ]:
df_q2 = visit_mapping_data(df_q2)
df_q3 = visit_mapping_data(df_q3)

In [ ]:
df_q2['主な来訪目的（16区分）'].value_counts()

In [ ]:
df_q3['主な来訪目的（16区分）'].value_counts()

In [ ]:
df = pd.concat([df_q2, df_q3],
               axis=0,
               ignore_index=True)
df.shape

## 主キーの設定

In [ ]:
def create_primary_key(df):
    df['userid'] = [f"{i:03d}" for i in range(1, len(df) + 1)]
    assert df['userid'].nunique() == len(df), "主キーが正しく設定されていません。"

    return df

In [ ]:
df = create_primary_key(df)
print(df.shape)
df['userid'].nunique()

## 都道府県コードと訪問地コードから名称を取得（既存の訪問名は削除）

In [ ]:
# 訪問地リスト
prefecture_df = pd.read_excel(config['prefectures_visited_list'])


def get_location_name_by_codes(df, prefecture_df):
    
    # 都道府県コードから都道府県名称を取得
    prefecture_mapping = pd.Series(prefecture_df['都道府県名称'].values,
                                   index=prefecture_df['都道府県コード']).to_dict()
    
    # 訪問地コードから訪問地名称を取得
    visit_location_mapping = pd.Series(prefecture_df['訪問地名称'].values,
                                       index=prefecture_df['訪問地コード']).to_dict()
    
    for i in range(1, 14):
        df[f'都道府県名称_{i}'] = df[f'都道府県コード_{i}'].map(prefecture_mapping)
        df[f'訪問地名称_{i}'] = df[f'訪問地コード_{i}'].map(visit_location_mapping)
    
    
    if df[f'都道府県コード_{i}'].notnull().any():
        assert df[f'都道府県名称_{i}'].dropna().isin(prefecture_mapping.values()).all(), f"都道府県名称_{i}に不正な値が含まれています"
    if df[f'訪問地コード_{i}'].notnull().any():
        assert df[f'訪問地名称_{i}'].dropna().isin(visit_location_mapping.values()).all(), f"訪問地名称_{i}に不正な値が含まれています"

    return df

In [ ]:
df = get_location_name_by_codes(df, prefecture_df)

In [ ]:
df.isna().sum()[df.isna().sum() > 0]

In [ ]:
df.shape

In [ ]:
# 訪問地名のカラムを削除する
for i in range(1, 14):  # 1から13までのカラムを削除
    visit_name_col = f'訪問地名_{i}'
    if visit_name_col in df.columns:
        df = df.drop(columns=[visit_name_col])

# 削除後のカラム存在確認
for i in range(1, 14):
    assert f'訪問地名_{i}' not in df.columns, f"カラム {i} が正しく削除されていません。"

## データを出力

In [ ]:
df.columns.to_list()

In [ ]:
cols = ['userid', '調査種別', '調査港（17区分）', '調査日', '入国日', '滞在日数（日）',
        '滞在日数（7区分）', '泊数（泊）', '入国空港・海港（2区分）',
        '入国空港・海港コード', '在留資格（6区分）', '数次有効ビザの発給FLG',
        '再入国許可FLG', '国籍・地域（21区分）', '国籍・地域', 
        '居住国・地域（2区分）', '居住国･地域（21区分）', '居住国・地域',
        '中国居住地（15区分）', '中国居住地その他【自由回答】',
        '性別（2区分）', '年齢区分（12区分）', '性年代（16区分）',
        '同行者（6区分）', '日本への来訪回数（8区分）', '前回来訪時期（6区分）',
        '年間来訪回数（前回まで）', '主な来訪目的（16区分）', '訪問地名_入国',
        '訪問地コード_入国', '都道府県コード_入国',
        '訪問地コード_1', '都道府県コード_1', '都道府県名称_1', '訪問地名称_1',
        '訪問地コード_2', '都道府県コード_2', '都道府県名称_2', '訪問地名称_2',
        '訪問地コード_3', '都道府県コード_3', '都道府県名称_3', '訪問地名称_3',
        '訪問地コード_4', '都道府県コード_4', '都道府県名称_4', '訪問地名称_4', 
        '訪問地コード_5', '都道府県コード_5', '都道府県名称_5', '訪問地名称_5',
        '訪問地コード_6', '都道府県コード_6', '都道府県名称_6', '訪問地名称_6',
        '訪問地コード_7', '都道府県コード_7', '都道府県名称_7', '訪問地名称_7',
        '訪問地コード_8','都道府県コード_8', '都道府県名称_8', '訪問地名称_8',
        '訪問地コード_9', '都道府県コード_9', '都道府県名称_9', '訪問地名称_9',
        '訪問地コード_10', '都道府県コード_10', '都道府県名称_10', '訪問地名称_10',
        '訪問地コード_11', '都道府県コード_11', '都道府県名称_11', '訪問地名称_11',
        '訪問地コード_12', '都道府県コード_12', '都道府県名称_12', '訪問地名称_12',
        '訪問地コード_13', '都道府県コード_13', '都道府県名称_13', '訪問地名称_13',
        '訪問地名_出国', '訪問地コード_出国', '都道府県コード_出国',
        '利用宿泊施設_入国', '利用宿泊施設_1', '利用宿泊施設_2', '利用宿泊施設_3',
        '利用宿泊施設_4', '利用宿泊施設_5', '利用宿泊施設_6',
        '利用宿泊施設_7', '利用宿泊施設_8', '利用宿泊施設_9', '利用宿泊施設_10',
        '利用宿泊施設_11', '利用宿泊施設_12', '利用宿泊施設_13']

In [ ]:
df[cols].to_excel(survey_2024_Q2andQ3_processed, index=False)

In [ ]:
df[cols].shape

In [ ]:
df[cols].head()